# Alerts

    Author: Jing Nghik
    Last Updated 2/9/2020

Purpose: This notebook is used to add device's to a certain group based on the machine tag.

## Requirements: 

### Scopes Required
- Alert.Read.All
- Alert.ReadWrite.All

### A client AppID/Secret or Device Code Authentication
[Proper MDATP API Token](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/exposed-apis-create-app-webapp)
        
    Microsoft Defender ATP exposes much of its data and actions through a set of programmatic APIs. Those APIs will help you automate work flows and innovate based on Microsoft Defender ATP capabilities.

## Set Variables and Import required modules and credentials 

In [ ]:
## Change these if required
$resource = "https://api.securitycenter.windows.com" ## resource Dont change since we want to query MDATP REST API Resource
$grant_type = "client_credentials" ## This is using a device_code flow (supports: client_credentials, device_code)

## Dont really need to change these unless necessary
$VerbosePreference = "ignore"   ## Set to 'continue to get verbose output'
## This will run a script to import the required modules and credentials stored in the "credential" file
$scriptPath = "~/Notebooks/PowerShell/custom-modules/import.ps1"; . $scriptPath  # this will run the script

## Authenticate with Graph API and get access Token for MDATP Resource

In [ ]:
## Get a token if authenticates properly. 

$authParams = @{
    tenantId = $credentials.tenantId
    client_id = $credentials.client_id
    client_secret = $credentials.client_secret
    resource = $resource  ## resource Dont change since we want to query MDATP REST API Resource
    grant_type = $grant_type  ## This is using a device_code flow (supports: client_credentials, device_code)
    #scope = "https://api.securitycenter.microsoft.com/Machine.ReadWrite.All" ## set the scope
};

## this will authenticate with Graph API and token will be stored in <tokenResponse> variable.
$token = (Invoke-GraphAuthentication -authParams $authParams)

## This is the authentication header for REST API Call
if ($token.access_token -ne $null) {
    $authHeader = @{ 
        'Content-Type' = 'application/json'
        Accept = 'application/json'
        Authorization = "Bearer $($token.access_token)" 
    }
}; $authHeader

## Gets the top 10 alerts

In [95]:
## Set the path you want to query
$endpoint = "api/alerts?top=10"
$method = "GET"

## The following commands will query the graph API and output the first 5 machines in table format
$uri = "$($resource)/$endpoint"; $uri
$Response = (Invoke-RestMethod -Method $method -Uri $uri -Headers $authHeader -ErrorAction Stop)
$Response


https://api.securitycenter.windows.com/api/alerts?top=10

@odata.context                                              value
--------------                                              -----
https://api.securitycenter.windows.com/api/$metadata#Alerts {}



## Create Alert by Reference 
- https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/create-alert-by-reference

In [99]:
$endpoint = "api/alerts/CreateAlertByReference"
$method = "POST"

$jsonBody = @"
{
  "reportId": "{ReportId of the associated event}",
  "eventTime": "2018-08-03T16:45:21.7115183Z",
  "machineId": "cb5402466f9427f3ea03c81c872ff9be0ce4b03b",
  "severity": "Low",
  "title": "test alert",
  "description": "test alert",
  "recommendedAction": "test alert",
  "category": "SuspiciousActivity"
}
"@

## The following commands will query the graph API and output the first 5 machines in table format
$uri = "$($resource)/$endpoint"; $uri
$Response = (Invoke-RestMethod -Method $method -Uri $uri -Headers $authHeader -body $jsonBody -ErrorAction Stop)
($Response | ConvertTO-JSON)

https://api.securitycenter.windows.com/api/alerts/CreateAlertByReference
Invoke-RestMethod: 
Line |
  19 |  … Response = (Invoke-RestMethod -Method $method -Uri $uri -Headers $aut …
     |                ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | {"error":{"code":"InvalidInput","message":"reportId is not in the correct format, expected long","target":"9d072c8d-5a95-4fe1-8818-297ce855a812"}}
{
  "@odata.context": "https://api.securitycenter.windows.com/api/$metadata#Alerts",
  "value": []
}
